<a href="https://colab.research.google.com/github/mikkeljais/sds_tddl_2020/blob/master/ex9_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

In [0]:
# load dataset
path2tox_data = '/content/drive/My Drive/Colab Notebooks/train.csv'
tox_df = pd.read_csv(path2tox_data)

tox_df['label'] = (tox_df.target>0.5).astype(int)

strat_sample_cols = list(tox_df.columns[3:23])+['physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white']
samples = []
n = 500
for col in strat_sample_cols:
    binary = pd.DataFrame((tox_df[col]>0.5).astype(int))
    samples+=[j for _,j in binary.groupby(col).apply(lambda x: x.sample(min(len(x),n//2))).index]
idx = list(set(samples))
df = tox_df.iloc[idx].copy()

sample = df.groupby('label').apply(lambda x: x.sample(500))
sample_texts = sample.comment_text.values
print(df.shape,tox_df.shape)

(11125, 46) (1804874, 46)


In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(tox_df['comment_text'], 
                                                    tox_df['label'], 
                                                    test_size=0.3, 
                                                    stratify=np.array(tox_df['label']), 
                                                    random_state=42)

val_dat = (np.array([i for i in x_test.values]), y_test.values)

In [0]:
#defining the basemodel
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
from keras_lr_multiplier import LRMultiplier
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
basetfmodel = tf.keras.models.Sequential()
basetfmodel.add(hub.KerasLayer(embed, input_shape=[], dtype=tf.string, trainable=False))
#Training is performed on the toxicity data
basetfmodel.compile(optimizer=tf.keras.optimizers.Adam('adam', {'Dense': 0.5, 'Output': 1.5}),  loss='sparse_categorical_crossentropy', )
basetfmodel.fit(x_train, y_train, epochs=10, batch_size=20000) #validation_data=(x_test, y_test), validation_data_batch_size=50000)

Epoch 1/10
64/64 [==============================] - 318s 5s/step - loss: 12.5852
Epoch 2/10
64/64 [==============================] - 319s 5s/step - loss: 12.5847
Epoch 3/10
 2/64 [..............................] - ETA: 2:37 - loss: 12.6065

KeyboardInterrupt: ignored

In [0]:
#start of by setting a threshold of 0.5
thresh = 0.5

#making the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(binary, np.array(tox_df['label']))

array([[1679244,  101778],
       [  19192,    4660]])

In [0]:
## using sklearn for splitting the dataset in train and validation set
## test set has been already provided

from sklearn.model_selection import train_test_split

train, val = train_test_split(tox_df,test_size = 0.05, random_state = 23)



In [0]:
## these are the identity columns, 

identity_columns = ['asian', 'atheist',
       'bisexual', 'black', 'buddhist', 'christian', 'female',
       'heterosexual', 'hindu', 'homosexual_gay_or_lesbian',
       'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
       'muslim', 'other_disability', 'other_gender',
       'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white']

y_identities_train = (train[identity_columns] >= 0.5).astype(int).values
y_identities_val = (val[identity_columns] >= 0.5).astype(int).values

In [0]:
#making the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(, np.array(tox_df['label']))

ValueError: ignored